# Transformation des données de pandémies - Partie 3: Normalisation

Ce notebook réalise la normalisation des données agrégées pour les préparer au chargement dans la base de données MySQL.

## 1. Configuration initiale

Importation des bibliothèques nécessaires et chargement des données agrégées.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime

# Configuration de la visualisation
%matplotlib inline
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Définition des chemins vers les fichiers sources
chemin_base = Path().resolve().parent.parent if 'notebooks' in str(Path().resolve()) else Path().resolve().parent
dossier_donnees_transformees = chemin_base / "donnees" / "transformees"

# Affichage du chemin
print(f"Dossier des données transformées: {dossier_donnees_transformees}")

In [ ]:
# Chargement des données agrégées
print("Chargement des données agrégées...")

try:
    # Chargement du jeu de données final
    fichier_donnees_final = dossier_donnees_transformees / "donnees_final.csv"
    if fichier_donnees_final.exists():
        donnees_final = pd.read_csv(fichier_donnees_final, parse_dates=["date"])
        print(f"Jeu de données final chargé: {donnees_final.shape[0]} lignes, {donnees_final.shape[1]} colonnes")
        print("\nAperçu du jeu de données final:")
        display(donnees_final.head())
    else:
        print(f"Impossible de charger le jeu de données final: fichier non trouvé")
    
    print("\nChargement des données agrégées terminé avec succès!")
except Exception as e:
    print(f"Erreur lors du chargement des données agrégées: {e}")

## 2. Création de la table calendrier

Création de la table calendrier à partir des dates présentes dans le jeu de données final.

In [ ]:
# Création de la table calendrier
print("Création de la table calendrier...")

try:
    # Extraction des dates uniques
    dates_uniques = donnees_final["date"].unique()
    print(f"Nombre de dates uniques: {len(dates_uniques)}")
    
    # Création de la table calendrier
    calendrier = pd.DataFrame({
        "id_date": range(1, len(dates_uniques) + 1),
        "date": sorted(dates_uniques),
    })
    
    # Ajout des colonnes jour, mois, année
    calendrier["jour"] = calendrier["date"].dt.day
    calendrier["mois"] = calendrier["date"].dt.month
    calendrier["annee"] = calendrier["date"].dt.year
    
    # Affichage de la table calendrier
    print(f"Table calendrier créée: {calendrier.shape[0]} lignes, {calendrier.shape[1]} colonnes")
    print("\nAperçu de la table calendrier:")
    display(calendrier.head())
    
    print("Création de la table calendrier terminée avec succès!")
except Exception as e:
    print(f"Erreur lors de la création de la table calendrier: {e}")

## 3. Création de la table localisation

Création de la table localisation à partir des pays présents dans le jeu de données final.

In [ ]:
# Création de la table localisation
print("Création de la table localisation...")

try:
    # Extraction des pays uniques
    pays_uniques = donnees_final["pays"].unique()
    print(f"Nombre de pays uniques: {len(pays_uniques)}")
    
    # Création de la table localisation
    localisation = pd.DataFrame({
        "id_localisation": range(1, len(pays_uniques) + 1),
        "pays": sorted(pays_uniques),
    })
    
    # Ajout de la colonne continent (à compléter manuellement ou avec une API)
    localisation["continent"] = np.nan
    
    # Affichage de la table localisation
    print(f"Table localisation créée: {localisation.shape[0]} lignes, {localisation.shape[1]} colonnes")
    print("\nAperçu de la table localisation:")
    display(localisation.head())
    
    print("Création de la table localisation terminée avec succès!")
except Exception as e:
    print(f"Erreur lors de la création de la table localisation: {e}")

## 4. Création de la table pandemie

Création de la table pandemie à partir des types de pandémies présents dans le jeu de données final.

In [ ]:
# Création de la table pandemie
print("Création de la table pandemie...")

try:
    # Extraction des types de pandémies uniques
    pandemies_uniques = donnees_final["type_pandemie"].unique()
    print(f"Nombre de types de pandémies uniques: {len(pandemies_uniques)}")
    
    # Création de la table pandemie
    pandemie = pd.DataFrame({
        "id_pandemie": range(1, len(pandemies_uniques) + 1),
        "nom_pandemie": sorted(pandemies_uniques),
    })
    
    # Ajout des colonnes description et date_debut
    descriptions = {
        "COVID-19": "Maladie à coronavirus 2019 causée par le SARS-CoV-2",
        "MPOX": "Maladie virale causée par le virus de la variole du singe (MPXV)"
    }
    dates_debut = {
        "COVID-19": "2019-12-01",
        "MPOX": "2022-05-01"
    }
    
    pandemie["description"] = pandemie["nom_pandemie"].map(descriptions)
    pandemie["date_debut"] = pandemie["nom_pandemie"].map(dates_debut)
    
    # Affichage de la table pandemie
    print(f"Table pandemie créée: {pandemie.shape[0]} lignes, {pandemie.shape[1]} colonnes")
    print("\nAperçu de la table pandemie:")
    display(pandemie)
    
    print("Création de la table pandemie terminée avec succès!")
except Exception as e:
    print(f"Erreur lors de la création de la table pandemie: {e}")

## 5. Création de la table donnees

Création de la table donnees à partir du jeu de données final et des tables de référence.

In [ ]:
# Création de la table donnees
print("Création de la table donnees...")

try:
    # Copie du jeu de données final
    donnees_df = donnees_final.copy()
    
    # Création d'un dictionnaire de correspondance pour les dates
    dict_dates = dict(zip(calendrier["date"], calendrier["id_date"]))
    
    # Création d'un dictionnaire de correspondance pour les pays
    dict_pays = dict(zip(localisation["pays"], localisation["id_localisation"]))
    
    # Création d'un dictionnaire de correspondance pour les pandémies
    dict_pandemies = dict(zip(pandemie["nom_pandemie"], pandemie["id_pandemie"]))
    
    # Remplacement des valeurs par les clés étrangères
    donnees_df["id_date"] = donnees_df["date"].map(dict_dates)
    donnees_df["id_localisation"] = donnees_df["pays"].map(dict_pays)
    donnees_df["id_pandemie"] = donnees_df["type_pandemie"].map(dict_pandemies)
    
    # Sélection des colonnes pertinentes
    donnees_df = donnees_df[[
        "id_date", "id_localisation", "id_pandemie", "cas_confirmes", "deces"
    ]]
    
    # Ajout d'un identifiant unique
    donnees_df.insert(0, "id_donnees", range(1, len(donnees_df) + 1))
    
    # Affichage de la table donnees
    print(f"Table donnees créée: {donnees_df.shape[0]} lignes, {donnees_df.shape[1]} colonnes")
    print("\nAperçu de la table donnees:")
    display(donnees_df.head())
    
    print("Création de la table donnees terminée avec succès!")
except Exception as e:
    print(f"Erreur lors de la création de la table donnees: {e}")

## 6. Sauvegarde des tables normalisées

Sauvegarde des tables normalisées pour l'étape de chargement.

In [ ]:
# Sauvegarde des tables normalisées
print("Sauvegarde des tables normalisées...")

try:
    # Sauvegarde de la table calendrier
    chemin_fichier = dossier_donnees_transformees / "calendrier.csv"
    calendrier.to_csv(chemin_fichier, index=False)
    print(f"Table calendrier sauvegardée: {chemin_fichier}")
    
    # Sauvegarde de la table localisation
    chemin_fichier = dossier_donnees_transformees / "localisation.csv"
    localisation.to_csv(chemin_fichier, index=False)
    print(f"Table localisation sauvegardée: {chemin_fichier}")
    
    # Sauvegarde de la table pandemie
    chemin_fichier = dossier_donnees_transformees / "pandemie.csv"
    pandemie.to_csv(chemin_fichier, index=False)
    print(f"Table pandemie sauvegardée: {chemin_fichier}")
    
    # Sauvegarde de la table donnees
    chemin_fichier = dossier_donnees_transformees / "donnees.csv"
    donnees_df.to_csv(chemin_fichier, index=False)
    print(f"Table donnees sauvegardée: {chemin_fichier}")
    
    print("\nSauvegarde des tables normalisées terminée avec succès!")
except Exception as e:
    print(f"Erreur lors de la sauvegarde des tables normalisées: {e}")

## Conclusion

Dans ce notebook, nous avons réalisé la normalisation des données agrégées pour les préparer au chargement dans la base de données MySQL. Les principales opérations ont été :

1. Création de la table calendrier à partir des dates présentes dans le jeu de données final
2. Création de la table localisation à partir des pays présents dans le jeu de données final
3. Création de la table pandemie à partir des types de pandémies présents dans le jeu de données final
4. Création de la table donnees à partir du jeu de données final et des tables de référence
5. Sauvegarde des tables normalisées pour l'étape de chargement

Les tables normalisées ont été sauvegardées dans des fichiers CSV pour l'étape de chargement, qui sera réalisée dans les notebooks de chargement.